In [80]:
import pandas as pd
import geocoder

In [81]:
education_df = pd.read_csv("university_df.csv")
education_df = education_df.drop("endYear", 1)
education_df[['startYear']] = education_df[['startYear']].fillna(value = 2017)

In [82]:
education_df = education_df[education_df.country.notnull()]

In [83]:
# education_df_empty_country = education_df[pd.isnull(education_df).any(axis=1)]

In [84]:
# school_country_dict = dict()
# names = education_df_empty_country['name'].unique()
# for name in names:
#     g = geocoder.google(name)
#     school_country_dict.update({name: g.country})

In [85]:
# for index, row in education_df_empty_country.iterrows():
#     country = school_country_dict.get(row['name'])
#     education_df.set_value(index, 'country', country)

# Insert actual ethnicity
# Insert type of the start year
### Type 1 > type 2 > type 3 ...
### Type 1 is farthest from now

In [86]:
def convert_actual_ethnicity(ethnicity):
    if ethnicity == 'White/Caucasian':
        return 'White'
    elif ethnicity == 'Hispanic/Latino':
        return 'Hispanic'
    elif ethnicity == 'Other/Multiracial':
        return 'Other'
    elif ethnicity == 'Black/African American':
        return 'Black'
    return ethnicity

In [87]:
education_df['actual_ethnicity'] = ''
education_df['type_year'] = ''
cand_infor_df = pd.read_csv("candilate_infor_crawed.csv")
for index, row in cand_infor_df.iterrows():
    cand_id = row['cand_id']
    actual_ethnicity = row['ethnicity']
    
    search_result = education_df[education_df.cand_id == cand_id]
    search_result = search_result.sort_values(['startYear'], ascending=True)
    
    i = 1
    for index_2, row_2 in search_result.iterrows():
        education_df.set_value(index_2, 'actual_ethnicity', convert_actual_ethnicity(actual_ethnicity))
        education_df.set_value(index_2, 'type_year', "type_"+str(i))
        i = i + 1

In [88]:
education_df = education_df[(education_df.type_year == 'type_1') \
                        | (education_df.type_year == 'type_2')\
                        | (education_df.type_year == 'type_3')]

In [89]:
# education_statistic = education_df[['type_year', 'country', 'actual_ethnicity', 'cand_id']].groupby(['type_year', 'country', 'actual_ethnicity']).agg(['count'])

In [90]:
# education_statistic.to_csv("education_statistic.csv")

# Statistic education

In [91]:
country_eth = pd.read_csv("country_ethnicity.csv", na_values='', keep_default_na = False)

In [92]:
education_df['predict_score'] = 0.0
education_df.head()

,cand_id,country,name,startYear,actual_ethnicity,type_year,predict_score
0,0,US,b'Foothill College',2017.0,White,type_3,0.0
1,0,CZ,b'ASEM',2002.0,White,type_2,0.0
2,0,US,b'MSU',2001.0,White,type_1,0.0
4,1,CN,b'Lanzhou University',2017.0,Asian,type_3,0.0
5,1,CN,b'Lanzhou University',1999.0,Asian,type_1,0.0


In [93]:
for index, row in education_df.iterrows():
    country_ethinicity = \
        country_eth[country_eth.country_code == row['country']].iloc[0]
    education_df.set_value(index, 'predict_score', \
                           country_ethinicity[row['actual_ethnicity']])

In [95]:
education_df[['predict_score']] = education_df[['predict_score']].fillna(value = 0)

In [103]:
len(education_df['cand_id'].value_counts())

965

In [110]:
education_df[(education_df.type_year == 'type_2') & (education_df.predict_score > 0)].mean()

cand_id          1300.979540
startYear        2007.828645
predict_score       0.806969
dtype: float64

In [114]:
education_df[(education_df.type_year == 'type_2') & (education_df.predict_score > 0)].mean()

cand_id          1300.979540
startYear        2007.828645
predict_score       0.806969
dtype: float64

In [117]:
education_df[(education_df.type_year == 'type_3') & (education_df.predict_score > 0)]

,cand_id,country,name,startYear,actual_ethnicity,type_year,predict_score
0,0,US,b'Foothill College',2017.0,White,type_3,0.75
4,1,CN,b'Lanzhou University',2017.0,Asian,type_3,1.00
7,2,US,b'Udacity',2017.0,White,type_3,0.75
13,4,US,b'San Francisco State University',2002.0,Hispanic,type_3,0.05
22,6,US,b'Central Michigan University',2000.0,White,type_3,0.75
29,8,US,b'Drexel University',2000.0,White,type_3,0.75
33,9,US,"b'University of California, Santa Cruz'",2002.0,Asian,type_3,0.03
41,10,US,b'Old Dominion University',1988.0,White,type_3,0.75
42,11,US,b'University of Michigan',2017.0,White,type_3,0.75
53,13,US,b'Arizona State University',2007.0,Asian,type_3,0.03
